<a href="https://colab.research.google.com/github/arthurcaique/ptfin/blob/analise_textual_arthur/textual_data_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 19.8MB/s 
  Running setup.py bdist_wheel for PyDrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
import json
from os import listdir
from os.path import join as join_path

import tarfile
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def _get_pet_info(petID: str, file: dict, tab_data: pd.DataFrame):
    entities = [{'entity': entity['name'], 'type': entity['type'], 'salience': 
                 entity['salience']} for entity in file['entities']]
    
    sentences = [{'sentence': sentence['text']['content'], 
                  'sentiment_score': sentence['sentiment']['score'], 
                  'sentiment_magnitude': sentence['sentiment']['magnitude']} 
                 for sentence in file['sentences']]
    
    document_sentiment = file['documentSentiment']
    language = file['language']
    
    pet_infos = tab_data[tab_data['PetID'] 
                         == petID][['AdoptionSpeed', 'Type']].values[0]
    
    adoption_speed = pet_infos[0]
    type_ = pet_infos[1]
    
    return {'entities': entities, 'sentences': sentences, 
            'document_sentiment': document_sentiment,
    'language': language, 'adoption_speed': adoption_speed, 'type': type_}

In [0]:
download = drive.CreateFile({'id': '18F3BZQooua_l6_66XwdSbfg9wY53FJBu'})
download.GetContentFile('train_sentiment.tar')

In [0]:
tar = tarfile.open('train_sentiment.tar', "r:")
tar.extractall()

files = {filename.replace('.json', '').split('/')[-1]: json.load(open(filename))
         for filename in tar.getnames() if filename.endswith('.json')}

tar.close()

In [0]:
download2 = drive.CreateFile({'id': '1j9eszo9HWocomdOQ7VztqtM1eRN1u6wL'})
download2.GetContentFile('train.csv')

tab_data = pd.read_csv('train.csv', sep = ',', encoding='utf-8')
tab_data = tab_data[['PetID', 'AdoptionSpeed', 'Type']]

In [0]:
pets_infos = [_get_pet_info(petID=petID, file=file, tab_data=tab_data) for petID, file in files.items()]
pets_infos_df = pd.DataFrame(pets_infos)

In [0]:
#ToDo verificar se os valores das keys nao utilizadas do json sao sempre todas vazias mesmo
#ToDo pesquisar como foi feito o classificacao do score de sentimento das frases e texto
#ToDo analytics: geral, por tipo de pet